In [1]:
# import sys
# sys.path.insert(0, '/home/esrf/james1997a/Code/ImageD11')

In [2]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "0"

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platforms", 'cpu')


from ImageD11.parameters import AnalysisSchema
from ImageD11.columnfile import columnfile
import ImageD11.transform, ImageD11.gv_general

In [3]:
pars = AnalysisSchema.from_default().geometry_pars_obj

In [4]:
pars.get_parameters()

{'chi': 0.0,
 'distance': 152736.55305695778,
 'fit_tolerance': 0.05,
 'min_bin_prob': 1e-05,
 'no_bins': 10000,
 'o11': -1,
 'o12': 0,
 'o21': 0,
 'o22': -1,
 'omegasign': 1.0,
 't_x': 0,
 't_y': 0,
 't_z': 0,
 'tilt_x': 0.0,
 'tilt_y': 0.0,
 'tilt_z': 0.0,
 'wavelength': 0.2845704,
 'wedge': 0.0,
 'weight_hist_intensities': 0,
 'y_center': 1049.9295061162281,
 'y_size': 75.0,
 'z_center': 1116.4472483389864,
 'z_size': 75.0}

In [5]:
pars.set('tilt_x', 0.00123)
pars.set('tilt_y', -0.0345)
pars.set('tilt_z', 0.02)
pars.set('chi', 1)
pars.set('wedge', -3)
pars.set('t_x', 1)
pars.set('t_y', 2)
pars.set('t_z', 3)

In [6]:
nrows = 100_000

fc = np.random.random(nrows) * 2048
sc = np.random.random(nrows) * 2048
om = np.random.random(nrows) * 360

In [7]:
cf = columnfile(new=True)
cf.nrows = nrows

cf.addcolumn(fc, 'fc')
cf.addcolumn(sc, 'sc')
cf.addcolumn(om, 'omega')

In [8]:
cf.parameters = pars

In [9]:
cf.updateGeometry()

In [10]:
kxyz_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))
cf.addcolumn(kxyz_id11[0], 'kx')
cf.addcolumn(kxyz_id11[1], 'ky')
cf.addcolumn(kxyz_id11[2], 'kz')

In [11]:
cf.titles

['fc',
 'sc',
 'omega',
 'xl',
 'yl',
 'zl',
 'tth',
 'eta',
 'ds',
 'gx',
 'gy',
 'gz',
 'kx',
 'ky',
 'kz']

In [12]:
# in my code, this is:
# (sc, fc): v_det
# omega: omega
# (xl, yl, zl): peak_lab
# (tth, eta): (tth, eta)
# (gx, gy, gz): q_sample
# (kx, ky, kz): q_lab

In [13]:
gvecs = np.column_stack([cf.gx, cf.gy, cf.gz])

In [14]:
import transform as mytrans

In [15]:
import importlib
importlib.reload(mytrans)

<module 'transform' from '/home/esrf/james1997a/Code/Anri/anri/sandbox/transform.py'>

In [16]:
# incident wavevector, normalised
k_in = mytrans._scale_norm_k(np.array([1., 0, 0]), pars.get('wavelength'))
k_in

Array([3.51406893, 0.        , 0.        ], dtype=float64)

In [17]:
origin_sample = jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z')))
origin_lab = mytrans.sample_to_lab(origin_sample, cf.omega, pars.get('wedge'), pars.get('chi'))

In [18]:
det_trans, beam_cen_shift, x_distance_shift = mytrans.detector_transforms(pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                                                          pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                                                          pars.get('tilt_x'),
                                                                          pars.get('distance'),
                                                                          pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

# Lab <-> Detector

In [19]:
v_lab_me = mytrans.det_to_lab(cf.sc, cf.fc, det_trans, beam_cen_shift, x_distance_shift)

In [20]:
assert np.allclose(v_lab_me[:, 0], cf.xl)
assert np.allclose(v_lab_me[:, 1], cf.yl)
assert np.allclose(v_lab_me[:, 2], cf.zl)

In [21]:
v_det_me = mytrans.lab_to_det(cf.xl, cf.yl, cf.zl, det_trans, beam_cen_shift, x_distance_shift)

In [22]:
assert np.allclose(v_det_me[:, 0], cf.sc)
assert np.allclose(v_det_me[:, 1], cf.fc)

# Sample <-> Lab

In [23]:
# map id11 k-vectors (q_lab) into sample frame (q_sample)

In [24]:
q_sample_me = mytrans.q_lab_to_q_sample(kxyz_id11.T, cf.omega, pars.get('wedge'), pars.get('chi'))

In [25]:
assert np.allclose(q_sample_me[:, 0], cf.gx)
assert np.allclose(q_sample_me[:, 1], cf.gy)
assert np.allclose(q_sample_me[:, 2], cf.gz)

In [26]:
q_lab_me = mytrans.sample_to_lab(gvecs, cf.omega, pars.get('wedge'), pars.get('chi'))

In [27]:
assert np.allclose(q_lab_me[:, 0], cf.kx)
assert np.allclose(q_lab_me[:, 1], cf.ky)
assert np.allclose(q_lab_me[:, 2], cf.kz)

In [28]:
q_sample_id11 = ImageD11.transform.compute_g_from_k(kxyz_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

assert np.allclose(q_sample_me, q_sample_id11.T)

In [29]:
rvec = np.random.random((3, nrows))
assert np.allclose(ImageD11.transform.compute_g_from_k(rvec, cf.omega, pars.get('wedge'), pars.get('chi')).T, mytrans.lab_to_sample(rvec.T, cf.omega, pars.get('wedge'), pars.get('chi')))

In [30]:
assert np.allclose(ImageD11.gv_general.k_to_g(rvec, -cf.omega, post=(mytrans.chimat(pars.get('chi')) @ mytrans.wedgemat(pars.get('wedge')))).T,ImageD11.transform.compute_g_from_k(rvec, cf.omega, pars.get('wedge'), pars.get('chi')).T)

# k_out

In [31]:
k_out_me_1 = mytrans.q_lab_to_k_out(kxyz_id11.T, k_in)

In [32]:
# should all have length (1/wavelength)
assert jnp.allclose(jnp.linalg.norm(k_out_me_1, axis=1) * pars.get('wavelength'), 1)

In [33]:
k_out_me_2 = mytrans.peak_lab_to_k_out(np.column_stack((cf.xl, cf.yl, cf.zl)), origin_lab, pars.get('wavelength'))

In [34]:
assert np.allclose(k_out_me_1, k_out_me_2)

In [35]:
# unit_vec_lab_me = mytrans.k_out_to_unit_vec_lab(k_out_me_1, origin_lab, pars.get('wavelength'))

In [36]:
# assert jnp.allclose(jnp.linalg.norm(unit_vec_lab_me, axis=1), 1)

In [37]:
sc_me, fc_me = mytrans.q_lab_to_det(kxyz_id11.T, cf.omega, origin_lab, k_in, pars.get('wavelength'), det_trans, beam_cen_shift, x_distance_shift)

In [38]:
sc_me

Array([1336.96788693,  643.76846353, 1840.83668033, ...,  557.48565273,
       1600.94776762,  991.32550757], dtype=float64)

In [39]:
assert np.allclose(sc_me, cf.sc)
assert np.allclose(fc_me, cf.fc)

# Full pipeline test: sc, fc, omega, origins to g-vectors

In [40]:
q_sample_me = mytrans.det_to_q_sample(cf.sc, cf.fc, cf.omega, origin_lab, k_in,
                                    pars.get('wedge'), pars.get('chi'), pars.get('wavelength'),
                                    det_trans, beam_cen_shift, x_distance_shift)

In [41]:
q_sample_me

Array([[ 0.76025571, -1.09013385, -0.38826776],
       [ 0.98409847,  0.13449682,  0.74606823],
       [ 0.01020778, -0.1772775 , -1.1753954 ],
       ...,
       [-0.05858302,  0.38517618,  0.92113375],
       [ 1.40359664, -0.54625387, -0.78423062],
       [-0.03618989, -0.66442137,  0.21831943]], dtype=float64)

In [42]:
gvecs

array([[ 0.76025571, -1.09013385, -0.38826776],
       [ 0.98409847,  0.13449682,  0.74606823],
       [ 0.01020778, -0.1772775 , -1.1753954 ],
       ...,
       [-0.05858302,  0.38517618,  0.92113375],
       [ 1.40359664, -0.54625387, -0.78423062],
       [-0.03618989, -0.66442137,  0.21831943]])

In [43]:
assert np.allclose(q_sample_me, gvecs)

In [44]:
[sc1_me, sc2_me], [fc1_me, fc2_me], [omega1_me, omega2_me], valid_me = mytrans.q_sample_to_det(gvecs, origin_lab, k_in, 
                                                                                               pars.get('wedge'), pars.get('chi'), pars.get('wavelength'),
                                                                                               det_trans, beam_cen_shift, x_distance_shift)

In [45]:
sc1_me[valid_me]

Array([1368.79579169,  658.17104065, 1844.14687569, ...,  562.08873979,
       1643.56649903,  991.32550757], dtype=float64)

In [46]:
sc2_me[valid_me]

Array([1336.96788693,  643.76846353, 1840.83668033, ...,  557.48565273,
       1600.94776762,  979.15271086], dtype=float64)

In [47]:
cf.sc

array([1336.96788693,  643.76846353, 1840.83668033, ...,  557.48565273,
       1600.94776762,  991.32550757])

In [48]:
cf.omega

array([156.12607561,  97.3830381 , 228.66194864, ...,  20.72478642,
       125.5284263 , 356.15071381])

In [49]:
omega1_me % 360

Array([314.19515441, 247.15696627, 304.85186788, ..., 142.08387954,
       277.10651945, 356.15071381], dtype=float64)

In [50]:
assert np.all(np.isclose(cf.sc, sc1_me[valid_me]) | np.isclose(cf.sc, sc2_me[valid_me]))
assert np.all(np.isclose(cf.fc, fc1_me[valid_me]) | np.isclose(cf.fc, fc2_me[valid_me]))
assert np.all(np.isclose(cf.omega, omega1_me[valid_me]  % 360) | np.isclose(cf.omega, omega2_me[valid_me] % 360))

# Lab <-> tth, eta, omega

In [51]:
tth_id11, eta_id11 = ImageD11.transform.compute_tth_eta_from_xyz(np.stack((cf.xl, cf.yl, cf.zl)), cf.omega, **pars.parameters)

In [52]:
tth_me, eta_me = mytrans.peak_lab_to_tth_eta(jnp.column_stack((cf.xl, cf.yl, cf.zl)),
                                             origin_lab,
                                             k_in,
                                             pars.get('wavelength'))

In [53]:
tth_me, eta_me

(Array([22.72420051, 20.36128096, 19.47553605, ..., 16.3625115 ,
        27.96345486, 11.43725108], dtype=float64),
 Array([-105.01344842,  -50.65050748, -173.63312886, ...,  -19.06691   ,
        -116.61432002,   72.39981864], dtype=float64))

In [54]:
tth_id11, eta_id11

(array([22.72420051, 20.36128096, 19.47553605, ..., 16.3625115 ,
        27.96345486, 11.43725108]),
 array([-105.01344842,  -50.65050748, -173.63312886, ...,  -19.06691   ,
        -116.61432002,   72.39981864]))

In [55]:
assert np.allclose(tth_me, tth_id11)
assert np.allclose(eta_me, eta_id11)

In [56]:
fc_id11, sc_id11 = ImageD11.transform.compute_xyz_from_tth_eta(cf.tth, cf.eta, cf.omega, **pars.parameters)

In [57]:
assert np.allclose(fc_id11, cf.fc)
assert np.allclose(sc_id11, cf.sc)

In [58]:
sc_me, fc_me = mytrans.tth_eta_omega_to_det(cf.tth, cf.eta, cf.omega, origin_lab, k_in,
                                      pars.get('wedge'), pars.get('chi'), pars.get('wavelength'),
                                      det_trans, beam_cen_shift, x_distance_shift)

In [59]:
assert np.allclose(sc_me, cf.sc)
assert np.allclose(fc_me, cf.fc)

In [60]:
t_id11 = ImageD11.transform.compute_grain_origins(cf.omega, pars.get('wedge'), pars.get('chi'), cf.xl, cf.yl, cf.zl)

In [61]:
t_me = mytrans.sample_to_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, pars.get('wedge'), pars.get('chi'))

In [62]:
assert np.allclose(t_id11, t_me.T)

In [63]:
%%time

tth_id11, (eta1_id11, eta2_id11), (omega1_id11, omega2_id11) = ImageD11.transform.uncompute_g_vectors(gvecs.T,
                                                                                                      pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 464 ms, sys: 41.8 ms, total: 506 ms
Wall time: 35 ms


In [64]:
[k1_me, k2_me], [omega1_me, omega2_me], valid = mytrans.q_sample_to_q_lab(gvecs, k_in, pars.get('wedge'), pars.get('chi'))

In [65]:
assert np.allclose(omega1_id11, omega1_me)
assert np.allclose(omega2_id11, omega2_me)

In [66]:
%%time

tth_me, (eta1_me, eta2_me), (omega1_me, omega2_me), valid = mytrans.q_sample_to_tth_eta_omega(gvecs, k_in,
                                                                                              pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )

CPU times: user 244 ms, sys: 16.5 ms, total: 261 ms
Wall time: 187 ms


In [67]:
omega1_me

Array([ -45.80484559, -112.84303373,  -55.14813212, ...,  142.08387954,
        -82.89348055,   -3.84928619], dtype=float64)

In [68]:
omega1_id11

array([ -45.80484559, -112.84303373,  -55.14813212, ...,  142.08387954,
        -82.89348055,   -3.84928619])

In [69]:
eta1_id11

array([107.01619255,  52.65325161, 175.63587299, ...,  21.06965413,
       118.61706416,  72.39981864])

In [70]:
eta1_me

Array([107.01619255,  52.65325161, 175.63587299, ...,  21.06965413,
       118.61706416,  72.39981864], dtype=float64)

In [71]:
cf.eta

array([-105.01344842,  -50.65050748, -173.63312886, ...,  -19.06691   ,
       -116.61432002,   72.39981864])

In [72]:
eta1_me[valid]

Array([107.01619255,  52.65325161, 175.63587299, ...,  21.06965413,
       118.61706416,  72.39981864], dtype=float64)

In [73]:
cf.eta[valid]

array([-105.01344842,  -50.65050748, -173.63312886, ...,  -19.06691   ,
       -116.61432002,   72.39981864])

In [74]:
assert np.allclose(omega1_id11, omega1_me)
assert np.allclose(omega2_id11, omega2_me)
assert np.allclose(tth_id11, tth_me)
assert np.allclose(eta1_id11, eta1_me)
assert np.allclose(eta2_id11, eta2_me)


In [75]:
t_id11 = ImageD11.transform.compute_grain_origins(cf.omega, pars.get('wedge'), pars.get('chi'), cf.xl, cf.yl, cf.zl)

In [76]:
t_me = mytrans.sample_to_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, pars.get('wedge'), pars.get('chi'))

In [77]:
t_id11.shape

(3, 100000)

In [78]:
t_me.shape

(100000, 3)

In [79]:
assert np.allclose(t_id11, t_me.T)

In [80]:
%%time

tth_id11, (eta1_id11, eta2_id11), (omega1_id11, omega2_id11) = ImageD11.transform.uncompute_g_vectors(gvecs.T, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 534 ms, sys: 47.5 ms, total: 581 ms
Wall time: 36.4 ms


In [81]:
tth_id11

array([22.72420051, 20.36128096, 19.47553605, ..., 16.3625115 ,
       27.96345486, 11.43725108])

In [82]:
eta1_me

Array([107.01619255,  52.65325161, 175.63587299, ...,  21.06965413,
       118.61706416,  72.39981864], dtype=float64)

In [83]:
eta1_id11

array([107.01619255,  52.65325161, 175.63587299, ...,  21.06965413,
       118.61706416,  72.39981864])

In [84]:
# test k vector computation
k_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))

In [85]:
k_id11

array([[-0.27277957, -0.2195684 , -0.2010612 , ..., -0.14232505,
        -0.41027859, -0.06978077],
       [ 1.31113245,  0.94548975,  0.12992442, ...,  0.32339279,
         1.4731815 , -0.66420259],
       [-0.35164674,  0.77523922, -1.16437962, ...,  0.9356499 ,
        -0.73817493,  0.21069979]])

In [86]:
k_me = mytrans.tth_eta_to_q_lab(cf.tth, cf.eta,k_in, pars.get('wavelength'), )

In [87]:
k_me

Array([[-0.27277957,  1.31113245, -0.35164674],
       [-0.2195684 ,  0.94548975,  0.77523922],
       [-0.2010612 ,  0.12992442, -1.16437962],
       ...,
       [-0.14232505,  0.32339279,  0.9356499 ],
       [-0.41027859,  1.4731815 , -0.73817493],
       [-0.06978077, -0.66420259,  0.21069979]], dtype=float64)

In [88]:
k_id11.T

array([[-0.27277957,  1.31113245, -0.35164674],
       [-0.2195684 ,  0.94548975,  0.77523922],
       [-0.2010612 ,  0.12992442, -1.16437962],
       ...,
       [-0.14232505,  0.32339279,  0.9356499 ],
       [-0.41027859,  1.4731815 , -0.73817493],
       [-0.06978077, -0.66420259,  0.21069979]])

In [89]:
k_id11.shape

(3, 100000)

In [90]:
k_me.shape

(100000, 3)

In [91]:
assert np.allclose(k_id11.T, k_me)

In [92]:
# test computation of angles for g-vectors

W =  mytrans.wedgemat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))
post = W @ C

oms_id11 = ImageD11.gv_general.g_to_k(gvecs.T, pars.get('wavelength'), axis=[0,0,-1], pre=None, post=post)

In [93]:
oms_id11

(array([ -45.80484559, -112.84303373,  -55.14813212, ...,  142.08387954,
         -82.89348055,   -3.84928619]),
 array([ 156.12607561,   97.3830381 , -131.33805136, ...,   20.72478642,
         125.5284263 , -169.81046043]),
 array([ True,  True,  True, ...,  True,  True,  True]))

In [94]:
oms_me = mytrans.omega_solns(gvecs, k_in, np.array([0,0,1]),  pars.get('wedge'), pars.get('chi'))

In [95]:
oms_me

(Array([ -45.80484559, -112.84303373,  -55.14813212, ...,  142.08387954,
         -82.89348055,   -3.84928619], dtype=float64),
 Array([ 156.12607561,   97.3830381 , -131.33805136, ...,   20.72478642,
         125.5284263 , -169.81046043], dtype=float64),
 Array([ True,  True,  True, ...,  True,  True,  True], dtype=bool))

In [96]:
assert np.allclose(oms_id11[0], oms_me[0])
assert np.allclose(oms_id11[1], oms_me[1])
assert np.allclose(oms_id11[2], oms_me[2])

In [97]:
# now use k vectors to test g_from_k

In [98]:
# test k vector computation
k_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))
g_id11 = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [99]:
g_id11

array([[ 0.76025571,  0.98409847,  0.01020778, ..., -0.05858302,
         1.40359664, -0.03618989],
       [-1.09013385,  0.13449682, -0.1772775 , ...,  0.38517618,
        -0.54625387, -0.66442137],
       [-0.38826776,  0.74606823, -1.1753954 , ...,  0.92113375,
        -0.78423062,  0.21831943]])

In [100]:
g_me = mytrans.lab_to_sample(k_id11.T, cf.omega, pars.get('wedge'), pars.get('chi'))

In [101]:
g_me

Array([[ 0.76025571, -1.09013385, -0.38826776],
       [ 0.98409847,  0.13449682,  0.74606823],
       [ 0.01020778, -0.1772775 , -1.1753954 ],
       ...,
       [-0.05858302,  0.38517618,  0.92113375],
       [ 1.40359664, -0.54625387, -0.78423062],
       [-0.03618989, -0.66442137,  0.21831943]], dtype=float64)

In [102]:
assert np.allclose(g_id11.T, g_me)

In [103]:
g_id11_trans = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [104]:
g_id11_trans

array([[ 0.76025571,  0.98409847,  0.01020778, ..., -0.05858302,
         1.40359664, -0.03618989],
       [-1.09013385,  0.13449682, -0.1772775 , ...,  0.38517618,
        -0.54625387, -0.66442137],
       [-0.38826776,  0.74606823, -1.1753954 , ...,  0.92113375,
        -0.78423062,  0.21831943]])

In [105]:
# C @ W -Z works

W =  mytrans.wedgemat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))

post = C @ W

g_id11_gvgeneral = ImageD11.gv_general.k_to_g(k_id11, cf.omega, axis=np.array([0., 0., -1]), pre=None, post=post)

In [106]:
g_id11_gvgeneral

array([[ 0.76025571,  0.98409847,  0.01020778, ..., -0.05858302,
         1.40359664, -0.03618989],
       [-1.09013385,  0.13449682, -0.1772775 , ...,  0.38517618,
        -0.54625387, -0.66442137],
       [-0.38826776,  0.74606823, -1.1753954 , ...,  0.92113375,
        -0.78423062,  0.21831943]])

In [107]:
assert np.allclose(g_id11_trans, g_id11_gvgeneral)

In [108]:
g_id11 = ImageD11.transform.compute_g_vectors(cf.tth, cf.eta, cf.omega, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

In [109]:
g_id11

array([[ 0.76025571,  0.98409847,  0.01020778, ..., -0.05858302,
         1.40359664, -0.03618989],
       [-1.09013385,  0.13449682, -0.1772775 , ...,  0.38517618,
        -0.54625387, -0.66442137],
       [-0.38826776,  0.74606823, -1.1753954 , ...,  0.92113375,
        -0.78423062,  0.21831943]])

In [110]:
assert np.allclose(gvecs, g_id11.T)

In [111]:
g_me = mytrans.tth_eta_omega_to_q_sample(cf.tth, cf.eta, cf.omega, k_in, pars.get('wedge'), pars.get('chi'), pars.get('wavelength'))

In [112]:
g_me

Array([[ 0.76025571, -1.09013385, -0.38826776],
       [ 0.98409847,  0.13449682,  0.74606823],
       [ 0.01020778, -0.1772775 , -1.1753954 ],
       ...,
       [-0.05858302,  0.38517618,  0.92113375],
       [ 1.40359664, -0.54625387, -0.78423062],
       [-0.03618989, -0.66442137,  0.21831943]], dtype=float64)

In [113]:
assert np.allclose(gvecs, g_me)

In [114]:
assert np.allclose(gvecs, mytrans.sample_to_lab(mytrans.lab_to_sample(gvecs, cf.omega, pars.get('wedge'), pars.get('chi')), cf.omega, pars.get('wedge'), pars.get('chi')))

In [115]:
# g-vectors to (tth, eta, omega)
tth, [eta1, eta_two], [omega1, omega2], valid = mytrans.q_sample_to_tth_eta_omega(gvecs, k_in,
                                                                                     pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )

In [116]:
tth

Array([22.72420051, 20.36128096, 19.47553605, ..., 16.3625115 ,
       27.96345486, 11.43725108], dtype=float64)

In [117]:
assert np.allclose(tth, cf.tth)
assert all(np.logical_or(np.isclose(cf.eta, eta1), np.isclose(cf.eta, eta_two)))
assert all(np.logical_or(np.isclose(cf.omega, omega1 % 360), np.isclose(cf.omega, omega2 % 360)))

In [118]:
# (tth, eta, omega) to g-vectors
gvecs_loop = mytrans.tth_eta_omega_to_q_sample(tth, eta1, omega1, k_in,
                                               pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )
assert np.allclose(gvecs_loop, gvecs)

In [119]:
# (tth, eta, omega) to g-vectors
gvecs_loop = mytrans.tth_eta_omega_to_q_sample(tth, eta_two, omega2, k_in,
                                               pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )
assert np.allclose(gvecs_loop, gvecs)

In [120]:
k_me = mytrans.peak_lab_to_q_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), origin_lab, k_in, pars.get('wavelength'), )

In [121]:
k_me

Array([[-0.27277957,  1.31113245, -0.35164674],
       [-0.2195684 ,  0.94548975,  0.77523922],
       [-0.2010612 ,  0.12992442, -1.16437962],
       ...,
       [-0.14232505,  0.32339279,  0.9356499 ],
       [-0.41027859,  1.4731815 , -0.73817493],
       [-0.06978077, -0.66420259,  0.21069979]], dtype=float64)

In [122]:
k_out = mytrans.q_lab_to_k_out(k_me, k_in)

In [123]:
origin_lab.shape

(100000, 3)

In [124]:
assert np.allclose(sc_me, cf.sc)

In [125]:
k_me.shape

(100000, 3)

In [126]:
origin_lab.shape

(100000, 3)

In [127]:
xyz_me = mytrans.q_lab_to_peak_lab(np.column_stack((cf.kx, cf.ky, cf.kz)), cf.omega, origin_lab, k_in,
                                   pars.get('wavelength'),
                                   det_trans, beam_cen_shift, x_distance_shift)

In [128]:
assert np.allclose(xyz_me, np.column_stack((cf.xl, cf.yl, cf.zl)))

In [129]:
xyz_me

Array([[152077.69105793,  61516.03302075, -16495.98311722],
       [150648.54919444,  43235.91720568,  35453.1593071 ],
       [154490.80156588,   6056.41879085, -54293.60219179],
       ...,
       [150999.93176253,  14484.9336883 ,  41904.99189748],
       [152543.43701204,  72403.7130393 , -36276.81109905],
       [153002.42427703, -29503.13245693,   9362.6124957 ]],      dtype=float64)

In [130]:
cf.xl

array([152077.69105793, 150648.54919444, 154490.80156588, ...,
       150999.93176253, 152543.43701204, 153002.42427703])

In [132]:
mytrans._lab_to_sample(np.array([1., 0, 0]), 90, 0, 0)

Array([ 6.123234e-17, -1.000000e+00,  0.000000e+00], dtype=float64)